<a href="https://colab.research.google.com/github/TheProffesional/Formacion-Bosonit/blob/main/KAFKA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nos desplazamos a la carpeta de instalación de kafka ( en 2 consolas diferentes) con el comando
 


```
cd kafka_2.11-2.1.0/
```



Luego en una consola ejecutamos el comando:



```
zookeeper-server-start.sh config/zookeeper.properties
```



Posteriormente en la otra:





```
 kafka-server-start.sh config/server.properties
```




 


Una vez arrancados zookeeper y kafka abrimos otra consola (una tercera) y en esta ejecutaremos los comandos indicados a continuación:



```
kafka-topics.sh
```



Vamos ahora a intentar crear un topic:



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic first_topic --create
```


Deberemos interpretar el mensaje de error, nos faltará indicar las particiones:



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic first_topic --create --partitions 3
```

Nos faltará un parámetro e interpretando el mensaje de error será el replication-factor:



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic first_topic --create --partitions 3 --
replication-factor 2
```

Interpretamos mensaje de error, el replication factor es mayor al número de brokers:



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic first_topic --create --partitions 3 --
replication-factor 1
```







Lista los topics:





```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --list
```



Describe el topic:


```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic first_topic --describe
```


Ahora crearemos otro topic:



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic second_topic --create --partitions 6 --
replication-factor 1
```


Listamos de nuevo los topics y comprobamos que ya hemos creado 2 (first_topic y
second_topic)




```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --list
```

Elimina topic marcado:



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic second_topic --delete
```






Lista de nuevo los topics, ahora deberá existir únicamente el “first_topic":



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --list
```
Ahora vamos a producir elementos para insertarlos en los topics de kafka:



```
kafka-console-producer.sh
```




Al ejecutar el comando escribiremos mensajes en la consola. Para salir pulsamos la combinación de teclas Ctrl+C.



```
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic first_topic
```




```
> Hola Mundo
> Aprendiendo Kafka
> Crtl+C 
```

Ahora le pasamos mas propiedades al producer, esta en especial espera la
sincronización de las replicas para realizar la confirmación, lo cual garantiza que el registro no se pierda mientras al menos una replica se 
mantenga levantado y en ejecución, esta (acks=all) es la garantía mas fuerte de
disponibilidad en la misma consola lanzamos el producer




```
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic first_topic --producerproperty acks=all
```




```
> Otro mensaje que es confirmado
> Seguimos aprendiendo
> Crtl+C
```





Ahora intentaremos escribir un mensaje en un topic que no existe, ¿Qué pasará?



```
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic new_topic
```



```
> Mensaje en un nuevo topic inexistente
> Otro mensaje en este topic nuevo
> Un último mensaje
> Crtl+C
```






El topic aunque no existe kafka lo crea



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --list
```



```
kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic new_topic --describe
```





Ahora veremos las formas de consumir los mensajes de kafka



```
kafka-console-consumer.sh
```

Exploramos todos los parámetros y opciones que tiene ese comando



```
kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic
```

No lee todo el topic sino que lee todos los nuevos mensajes que lleguen una vez
lanzado el consumidor, esa es la razón por la cual no ha leído los mensajes que habíamos escrito antes así que en otra consola (la cuarta) empezamos a escribir de nuevo en este topic para ver los mensajes .Este producer a diferencia de como hemos hecho anteriormente lo dejaremos ejecutando (NO utilizamos Ctrl+C)



```
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic first_topic
```



```
> Hola nuevo mensaje para el consumer
> Más mensajes de prueba
> arriba españa
```

Para leer todos los mensajes Cancelamos el consumer (utilizando Ctrl+C) y lo lanzamos de nuevo pero indicando que
lea los mensajes desde 
el principio del topic y así podremos ver todos los mensajes además de los que vayan llegando Aprovechamos de interactuar y enviar más mensajes con el producer y así ver como sonconsumidos todos los mensajes 
incluidos los recién enviados




```
kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --from-beginning
```







Finalmente detenemos tanto el consumer como el producer utilizando en cada una de estas consultas la combinación de teclas Ctrl+C.


Ahora veremos el uso de los consumers group donde el group es el equivalente a lo que sería el nombre de nuestra aplicación. Tiene a su vez la definición de grupo ya que consiste en un grupo de consumidores. 
Esta ha de ser la forma de realizar el consumo de datos desde kafka por las aplicaciones (por ejemplo spark streaming) cuando involucremos alto rendimiento, concurrencia, entre otras cosas.
En la consola donde estábamos ejecutando el consumer lanzamos el consumer que
forma parte de un consumer group



```
kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group
my-first-application
```

En la otra consola lanzamos de nuevo el producer para escribir en el first_topic y empezamos a mandar mensajes



```
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic first_topic
```

ahora abrimos otra consola (la quinta) con y lanzamos otro consumer con el mismo
grupo de consumo




```
kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group
my-first-application
```

Empezamos a lanzar mensajes desde la otra consola producer y vemos como los
consumidores van leyendo los mensajes, distribuyéndose estos mensajes entre los 2 consumidores.

Y podemos incluso crear otro consumer que forme parte del grupo y seguir lanzando mensajes y ver como los consumidores el grupo van leyendo los mensajes



## EJERCICIO 1

¿Qué pasaría si cancelamos (utilizando Ctrl+C) uno de los
consumidores (quedando 2) y seguimos
enviando mensajes por el producer?

## EJERCICIO 2

 ¿Qué pasaría si cancelamos otro de los consumidores (quedando ya
solo 1) y seguimos enviando
mensajes por el producer?

## EJERCICIO 3

¿Qué sucede si lanzamos otro consumidor pero está vez de un grupo
llamado my-second-application
leyendo el topic desde el principio (--from-beginning)?

## EJERCICIO 4

Cancela el consumidor y ¿Qué sucede si lanzamos de nuevo el
consumidor pero formando parte del
grupo my-second-application?¿Aparecen los mensajes desde el principio

## EJERCICIO 5

Cancela el consumer, a su vez aprovecha de enviar más mensajes
utilizando el producer y de nuevo
lanza el consumidor formando parte del grupo my-second_application ¿Cuál fue el
resultado?

Por último exploraremos otro comando


```
kafka-consumer-groups.sh
```

Una vez analizadas todas las opciones y argumentos que recibe, lanzamos el siguiente comando



```
kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --list
```

Podremos visualizar el offset por particiones



```
kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --describe --group my-second-application
```
Podemos también consultar la información del primer grupo de consumo (my-first-application).
A modo de prueba lanzamos de nuevo el consumer para el grupo my-first-application ya que puede que todavía haya 
mensajes que no hayamos leído, en mi caso al haber estado lanzando mensajes para
ver como los consumía el grupo my-second-application me arroja lo siguiente después de ejecutar el comando




```
kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group
my-first-application

```

Resetearemos el offset del consumer group
Para ello primero leeremos las opciones del comando y nos enfocaremos en —reset-offsets



```
kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --group my-first-application
--reset-offsets --to-earliest
```

Interpretamos el error y adecuamos nuestra sentencia



```
kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --group my-first-application
--reset-offsets --to-earliest --execute --topic first_topic
```



¿Qué pasa si reiniciamos el consumidor?



```
kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group
my-first-application

```



```
kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --list --describe --group
my-first-application
```

probamos otra opción para mover el cursor
Investigar en que consiste el shift-by



```
kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --group my-first-application
--reset-offsets --shift-by 2 --execute --topic first_topic
```

NOTA: Puede que tengamos valores diferentes en el caso de que hayamos enviado un
numero diferente de mensajes a kafka con los producer, pero lo importantes es que al invocar este comando deberíamos
recibir el warning



```
kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --group my-first-application
--reset-offsets --shift-by -2 --execute --topic first_topic
```

Podemos observar como se ha modificado el offset
Probamos de nuevo lanzar el consumidor asociado al consumer group my-firstapplication y vemos los mensajes que muestra 
de acuerdo a la modificación que realizamos del offset



```
kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic first_topic --group
my-first-application
```




## EJERCICIO FINAL



*   Crear topic llamado "topic_app" con 3 particiones y replication-factor = 1.
*   Crear un producir que inserte mensajes en el topic recien creado (topic_app).



*   Crear 2 consumer que forman parte de un grupo de consumo llamado "my_app".

*   Interactuar con los 3 elementos creando mensajes en el producir y visualizar como
estos son consumidos por los consumer


*   Aplicar los comandos necesarios para listar los topics, grupos de consumo, así
como describir cada uno de estos







